In [1]:
import pandas as pd
import numpy as np
import math 

In [2]:
plant_record = []
fault_record = []
plant_list = [1,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,21,23,24,27,28,29,30,31,32,33,34,35,36,39,40]

In [3]:
def read_file(plant_index,file_path):
    train_ori_name = file_path+"/PHM_train"+str(plant_index)+".csv"
    train_ori = pd.read_csv(train_ori_name,dtype = np.float64)
    test_ori_name = file_path+"PHM_test"+str(plant_index)+".csv"
    test_ori  = pd.read_csv(test_ori_name,dtype = np.float64)
    train_label_ori_name = file_path+"PHM_trainLabel"+str(plant_index)+".csv"
    train_label_ori = pd.read_csv( train_label_ori_name,dtype = np.float64)
    test_label_ori_name = file_path+"PHM_testLabel_viseble"+str(plant_index)+".csv"
    test_label_visible_ori = pd.read_csv(test_label_ori_name,dtype = np.float64)
    
    test_label_hidden_name = file_path+"PHM_testLabel_hidden"+str(plant_index)+".csv"
    test_label_hidden_ori = pd.read_csv(test_label_hidden_name,dtype = np.float64)
    return train_ori,test_ori,train_label_ori,test_label_visible_ori ,test_label_hidden_ori

In [4]:
def remove_na_dup(dataset):
    dataset = dataset.drop_duplicates()
    dataset = dataset.dropna()
    return dataset

In [5]:
def onehot_encode(train_ori,test_ori):
    train_onehot = pd.DataFrame( {"ini":train_ori["season"]})
    test_onehot = pd.DataFrame( {"ini":test_ori["season"]})

    for index in range(1,5):
        train_onehot[( "season_"+str(index) )] = (train_ori["season"] == index ).astype(int)
        test_onehot[( "season_"+str(index) )] = (test_ori["season"] == index ).astype(int)

    for index in range(1,13):
        train_onehot[( "month_"+str(index) )] = (train_ori["month"] == index ).astype(int)
        test_onehot[( "month_"+str(index) )] = (test_ori["month"] == index ).astype(int)

    for index in range(1,8):
        train_onehot[( "wday_"+str(index) )] = (train_ori["wday"] == index ).astype(int)
        test_onehot[( "wday_"+str(index) )] = (test_ori["wday"] == index ).astype(int)

    for index in range(1,25):
        train_onehot[( "hour_"+str(index) )] = (train_ori["hour"] == index ).astype(int)
        test_onehot[( "hour_"+str(index) )] = (test_ori["hour"] == index ).astype(int)

    train_onehot = train_onehot.drop("ini",1)
    test_onehot = test_onehot.drop("ini",1)
    return train_onehot,test_onehot

In [6]:
def standardize(dataset_train,dataset_test):
    b = (dataset_test - dataset_train.mean())/dataset_train.std()
    return b

In [7]:
def combine_addzero_feature(sensor,onehot,width):
    dataset = pd.concat([onehot,sensor],axis = 1)
    dataset.columns.values
    ##add padding
    for index in range( 13**2-dataset.shape[1] ):
        dataset[("zero_"+str(index))] = 0
    return dataset

In [8]:
def add_laged_feature(dataset,dataset_ori,section):
    train_laged = dataset[ section:(dataset.shape[0]-section)]
    train_laged.index = range(train_laged.shape[0])
    train_laged_time = dataset_ori.iloc[ section:(dataset.shape[0]-section),0 ]
    train_laged_time.index = range(train_laged_time.shape[0])


    
        ##before section
    for index in range(section):
        add_lag = dataset[index:(dataset.shape[0]-section*2+index)]
        add_lag.columns = add_lag.columns.values+"_-"+str(index)
        add_lag.index = range(add_lag.shape[0])
        train_laged = pd.concat([train_laged, add_lag],axis = 1)


        ##after section
    for index in range(section):
        add_lag = dataset[(section+index):(dataset.shape[0]-section+index)]
        add_lag.columns = add_lag.columns.values+"_+"+str(index)
        add_lag.index = range(add_lag.shape[0])
        train_laged = pd.concat([train_laged, add_lag],axis = 1)

            ##combine
    train_c = pd.concat([train_laged_time,train_laged],axis = 1)


    return train_c

In [9]:
def add_label(dataset,dataset_fault):
    ##add label
    dataset["end_label"] = 0
    for index in range( dataset_fault.shape[0] ):
        label_v = range( int(dataset_fault["endTime"][index]-4) ,int(dataset_fault["endTime"][index]+5) )    
        dataset.loc[dataset.time.isin(label_v),'end_label'] = 1

    return dataset        

In [10]:
##extract the 95bound from starttime
def extract_sec(dataset,data_label,bound95):
    init_time = data_label["startTime"][0]
    section = range(int(init_time+1),int(init_time+bound95))
    return_data = dataset.loc[dataset.time.isin(section),]
    
    for time in data_label["startTime"][1:]:
        section = range(int(time+1),int(time+bound95))
        section_data = dataset.loc[dataset.time.isin(section),]
        return_data = pd.concat([return_data,section_data], axis = 0)
    return return_data

In [11]:
def find_bound95(train_sub_fault,test_sub_fault):
    train_section = (train_sub_fault["endTime"] - train_sub_fault["startTime"]).tolist()
    test_section = (test_sub_fault["endTime"] - test_sub_fault["startTime"]).tolist()
    train_section.extend(test_section)
    all_section = sorted(train_section)
    bound = all_section[ math.floor(len(all_section)*95/100)]
    return bound

In [12]:
def wrire_record(record_testplant,record_testfault):
    f = open("test_plant_record.csv","w")
    for data in record_testplant:
        f.write( str(data)+",")
    f.close()

    f = open("test_fault_record.csv","w")
    for data in record_testfault:
        f.write( str(data)+",")
    f.close()

In [13]:
##main function
def do_preprocess(width,section,read_file_path,save_file_path):
    record_bound = []
    ##deal bound
    bound_count = 0
    ##deal bound
    for n_index in range(len(plant_list)):
        plant_index = plant_list[n_index]
        print("plant_index: %s  total: %s"%(n_index,len(plant_list)))
        ##for loop
        train_ori,test_ori,train_label_ori,test_label_visible_ori,test_label_hidden_ori = read_file(plant_index,read_file_path)
        train_ori = remove_na_dup(train_ori)
        test_ori = remove_na_dup(test_ori)
        train_sensor = train_ori.drop(["time","year","season","month","wday","hour"],1)
        test_sensor = test_ori.drop(test_ori.columns[:6],1)
        train_onehot,test_onehot = onehot_encode(train_ori,test_ori)
        test_sensor = standardize(train_sensor, test_sensor)
        train_sensor = standardize(train_sensor,train_sensor)
        train_basic = combine_addzero_feature(train_sensor,train_onehot,width)
        test_basic = combine_addzero_feature(test_sensor,test_onehot,width)
        train_combine = add_laged_feature(train_basic,train_ori,section)
        test_combine = add_laged_feature(test_basic,test_ori,section)
        #different faults
        for fault_index in train_label_ori["fault"].unique().astype(int):
            ##for loop
            
            train_sub = train_combine
            test_sub = test_combine
            train_sub_fault = train_label_ori[ train_label_ori["fault"] == fault_index ]
            train_sub_fault.index = range(train_sub_fault.shape[0])
            test_sub_fault = test_label_visible_ori[test_label_visible_ori["fault"] == fault_index ]
            test_sub_fault.index = range(test_sub_fault.shape[0])
            train_sub = add_label(train_sub,train_sub_fault)
            test_sub = add_label(test_sub,test_sub_fault)
            bound_real = find_bound95(train_sub_fault,test_sub_fault)
            bound95 = max(bound_real , 8)
            record_bound.append(int(bound_real) )
            print("fault: %s real_bound: %s" % (fault_index, bound_real))
            if bound95 < 20:
                bound95 = 20
            train_sec = extract_sec(train_sub,train_sub_fault,bound95)
            if(test_sub_fault.shape[0]>0):
                test_sec = extract_sec(test_sub,test_sub_fault,bound95)
                train_set = pd.concat([train_sec,test_sec],axis = 0)
                test_set = test_sub.loc[test_sub.time.isin(test_sec.time) == False,:]
            else:
                train_set = train_sec
                test_set = test_sub
            test_set = test_set.loc[test_set.end_label == 0,:]
            ##label the test set
            test_hidden_label = test_label_hidden_ori.loc[test_label_hidden_ori.fault == fault_index,:]
            test_hidden_label.index = range(test_hidden_label.shape[0])
            test_set = add_label(test_set, test_hidden_label)

            num_zero = (train_set.end_label == 0).sum()
            num_one  =  (train_set.end_label==1).sum()
            print( "plant_index: %s,fulat_index: %s, total: %s fault num: %s" %(plant_index,fault_index,len(plant_list),train_sub_fault.shape[0]))
            print("train_set: %s zero: %s one: %s bound: %s" % (train_set.shape[0], num_zero, num_one, bound95) )
            print("test_set: %s" % (test_set.shape[0]) )
            ##write csv
            train_set_name = save_file_path+"/end/end_trainset"+str(plant_index)+"_fault"+str(fault_index)+".csv"
            train_set.to_csv(train_set_name,index = False,sep = "," )

            test_set_name = save_file_path+"/end/end_testset"+str(plant_index)+"_fault"+str(fault_index)+".csv"
            test_set.to_csv(test_set_name,index = False,sep = "," )
            
            bound_count +=1
            
            f_name = save_file_path + "/record_bound.csv"
            f = open(f_name, "w")
            f.write(str(record_bound[0]))
            for data in record_bound[1:]:
                f.write( "," + str(data))
            f.close()
            


In [14]:
##parameters setting
width = 13
section = 4
read_file_path = "/media/joeytu/82F6C88FF6C8853F/PHM2017/PHM_rawdata/"
save_file_path = "/media/joeytu/82F6C88FF6C8853F/PHM2017/PHM_nosample/file"

##do_preprocess
do_preprocess(width,section,read_file_path,save_file_path) 

plant_index: 0  total: 31
fault: 1 real_bound: 20.0
plant_index: 1,fulat_index: 1, total: 31 fault num: 485
train_set: 11883 zero: 6744 one: 5139 bound: 20.0
test_set: 25701
fault: 2 real_bound: 42.0
plant_index: 1,fulat_index: 2, total: 31 fault num: 576
train_set: 23742 zero: 13719 one: 10023 bound: 42.0
test_set: 28484
fault: 3 real_bound: 45.0
plant_index: 1,fulat_index: 3, total: 31 fault num: 402
train_set: 17423 zero: 10793 one: 6630 bound: 45.0
test_set: 28888
fault: 4 real_bound: 24.0
plant_index: 1,fulat_index: 4, total: 31 fault num: 135
train_set: 2956 zero: 1493 one: 1463 bound: 24.0
test_set: 28888
fault: 5 real_bound: 7.0
plant_index: 1,fulat_index: 5, total: 31 fault num: 53
train_set: 3930 zero: 2174 one: 1756 bound: 20
test_set: 26203
plant_index: 1  total: 31
fault: 1 real_bound: 38.0
plant_index: 3,fulat_index: 1, total: 31 fault num: 534
train_set: 22830 zero: 16142 one: 6688 bound: 38.0
test_set: 21708
fault: 4 real_bound: 6.0
plant_index: 3,fulat_index: 4, total: